In [1]:
import twitterGraph as tg
import numpy as np
import random as rand
import tweepy
from config import tw_key, tw_secret
#graph = tg.twitterGraph()
auth = tweepy.AppAuthHandler(tw_key, tw_secret)
api = tweepy.API(auth)

In [2]:
lists = api.lists_all("CSPAN")

In [17]:
import json

In [ ]:
[x.name for x in lists]

In [5]:
congress = lists[16]
governors = lists[18]

In [6]:
politicians = []

In [7]:
for item in tweepy.Cursor(api.list_members, list_id=congress.id).items():
    politicians.append(item)

In [8]:
for item in tweepy.Cursor(api.list_members, list_id=governors.id).items():
    politicians.append(item)

In [9]:
politicians[0].__dir__()

['_api',
 '_json',
 'id',
 'id_str',
 'name',
 'screen_name',
 'location',
 'description',
 'url',
 'entities',
 'protected',
 'followers_count',
 'friends_count',
 'listed_count',
 'created_at',
 'favourites_count',
 'utc_offset',
 'time_zone',
 'geo_enabled',
 'verified',
 'statuses_count',
 'lang',
 'status',
 'contributors_enabled',
 'is_translator',
 'is_translation_enabled',
 'profile_background_color',
 'profile_background_image_url',
 'profile_background_image_url_https',
 'profile_background_tile',
 'profile_image_url',
 'profile_image_url_https',
 'profile_banner_url',
 'profile_link_color',
 'profile_sidebar_border_color',
 'profile_sidebar_fill_color',
 'profile_text_color',
 'profile_use_background_image',
 'has_extended_profile',
 'default_profile',
 'default_profile_image',
 'can_media_tag',
 'followed_by',
 'following',
 'follow_request_sent',
 'notifications',
 'translator_type',
 '__module__',
 'parse',
 'parse_list',
 'timeline',
 'friends',
 'followers',
 'follow',


In [40]:
k = 0
dat = []
for page in tweepy.Cursor(api.user_timeline,id=politicians[0].id_str, tweet_mode='extended', count=200).pages():
    dat += page
    k += 1
    if k >= 3:
        break


In [42]:
dat[-1].created_at

datetime.datetime(2019, 6, 5, 0, 25, 3)

In [12]:
temp = 0
for x in politicians:
    temp += x.followers_count

In [16]:
inFileName = "data/pol_json_data"

In [18]:
pols_data = []
with open(inFileName, "r") as f:
    for line in f:
        pols_data.append(json.loads(line))

In [26]:
[x["followers_count"] for x in pols_data]

[4643,
 2562,
 3065,
 3012,
 11911,
 4688,
 6051,
 9570,
 24956,
 6994,
 13163,
 22567,
 7513,
 525501]

In [24]:
525000/5000

105.0

In [10]:
def politician_scrape(pol_id):
    from math import ceil
    from time import sleep
    try:
        pol = api.get_user(id=pol_id)
    except:
        return None
    num_mins = ceil(max(pol.followers_count, pol.friends_count)/5000)
    k = 0
    statuses = []
    followers = []
    friends=[]
    fol_cur = tweepy.Cursor(api.followers_ids,id=pol.id_str, count=5000).pages()
    fr_cur = tweepy.Cursor(api.friends_ids,id=pol.id_str, count = 5000).pages()
    stat_cur = tweepy.Cursor(api.user_timeline,id=pol.id_str, tweet_mode="extended", count=200).pages()
    while k < num_mins:
        t_stat = [p._json for p in next(stat_cur)]
        for t in t_stat:
            t.pop('user')
        statuses += t_stat
        friends += next(fr_cur)
        followers += next(fol_cur)
        try:
            k += 1
            if k < num_mins:
                time.sleep(1)
        except:
            pass
    
    out = {
        'id':pol.id,
        'id_str':pol.id_str,
        'name':pol.name,
        'screen_name':pol.screen_name,
        'location':pol.location,
        'description':pol.description,
        'friends_count':pol.friends_count,
        'followers_count':pol.followers_count,
        'created_at':pol.created_at,
        'friends_ids':friends,
        'followers_ids':followers,
        'statuses':statuses
    }
    return out

In [43]:
tweets = []
for page in tweepy.Cursor(api.user_timeline,id=politicians[0].id_str, count=20, tweet_mode='extended').pages():
    tweets += page
    break

In [46]:
tweets[0]._json['truncated']

False

In [35]:
tweets[84]._json['truncated']

False

In [ ]:
for politician in politicians:
    if politician.id in democrats_list:
        politician['party'] = 'D'
    elif politician.id in republicans_list:
        politician['party'] = 'R'
    else:
        politician['party'] = 'I'


In [53]:
oldPoliticians = [graph.getUser(x.id) for x in politicians if graph.getUser(x.id)]

In [58]:
len(oldPoliticians)

121

In [47]:
max(4,1)

4

In [26]:
status._json['entities']

{'hashtags': [],
 'symbols': [],
 'user_mentions': [{'screen_name': 'HouseDemocrats',
   'name': 'House Democrats',
   'id': 43963249,
   'id_str': '43963249',
   'indices': [94, 109]}],
 'urls': [{'url': 'https://t.co/iS7mYJAhmz',
   'expanded_url': 'https://twitter.com/i/web/status/1194740892155994119',
   'display_url': 'twitter.com/i/web/status/1…',
   'indices': [116, 139]}]}

In [22]:
list(status.__dict__.keys())

['_api',
 '_json',
 'created_at',
 'id',
 'id_str',
 'text',
 'truncated',
 'entities',
 'source',
 'source_url',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'lang']

In [13]:
tweet = out['statuses'][0]

In [26]:
my_data = {
    'id_str':tweet['id_str'],
    'geo':tweet['geo'],
    'text':tweet['full_text']}

In [27]:
my_data

{'id_str': '1194362516165541888',
 'geo': None,
 'text': 'Congratulations to \u2066@FarmBureau\u2069 on celebrating your Centennial today! Your commitment to advocacy for our nation’s agriculture industry has been appreciated by the farmers in #PA12! https://t.co/jIg3qaVdfe'}

In [44]:
congr115 = pd.read_csv("C:/Users/ericj/Downloads/115th-Congress-House-seeds.csv", header = None, names=["name", "id"], dtype=str)

In [45]:
sen115 = pd.read_csv("C:/Users/ericj/Downloads/115th-Congress-Senate-seeds.csv", header = None, names=["name", "id"], dtype=str)

In [47]:
for p in list(pd.concat([congr115, sen115], axis=0).reset_index()["id"]):
    if p not in pol_ids:
        pol_ids.append(p)

In [48]:
len(pol_ids)

772

In [50]:
pols_sr = pd.Series(pol_ids)

In [54]:
f = open("data/politicians_list", "w")
for p in pol_ids:
    f.write(p + '\n')
f.close()